In [1]:
import string
import warnings
from configparser import ConfigParser

from easydict import EasyDict
from pytorch_lightning import seed_everything

from models.protoconv.comet_connector import CometConnector
from models.protoconv.data_visualizer import DataVisualizer
from utils import plot_html

warnings.simplefilter("ignore")
seed_everything(0)

config = ConfigParser()
config.read('../config.ini')
comet_config = EasyDict(config['cometml'])

comet_connector = CometConnector(comet_config.apikey, comet_config.projectname, comet_config.workspace, rel_path='..')


Global seed set to 0


In [2]:
experiments_7gram = {
    'imdb': '3952f03cdf294788a219421f539bd285',
    'amazon': '5211ae86ae4949468f1f7a1e495cc6b4',
    'yelp': '13051aa6d2f3480ca4ee11a5bd82f5ce',
    'rottentomatoes': '21be41b9bb304fbc9a7322775e78fe14',
    'hotel': 'f09b420d28654e7cbf8ec13326b0944d'
}

In [19]:
from nltk.corpus import stopwords

import string
import pandas as pd

pd.set_option('display.max_rows', 110)
pd.set_option('display.max_colwidth', 500)

for dataset, experiment_id in experiments_7gram.items():

    for fold_id in range(5):
        #
        # if dataset!='imdb' or fold_id!=2:
        #     continue

        plot_html(f'<h1>{dataset} {fold_id}</h1>')
        stopwords.words('english')
        prototypes_words = []

        comet_connector.set_experiment(experiment_id, fold=fold_id)
        comet_connector.get_model()

        model = comet_connector.model
        model.cuda()
        model.eval()
        model.conv1.conv1.padding = 0

        data_visualizer = DataVisualizer(model)

        words_matrix = model.prototype_tokens.tolist()
        prototypes_id_tokens_to_extend = []

        stoi = comet_connector.TEXT.vocab.stoi
        itos = comet_connector.TEXT.vocab.itos

        words_remove = [w for w in stopwords.words('english') + ["'m", "'re", "'s", '..', '...', '....'] + list(string.punctuation) if w not in ['not', "don't", 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",  'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",  'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't",   'wouldn', "wouldn't"]]
        df_data = []
        for proto_id, tokens_list in enumerate(words_matrix):
            if model.enabled_prototypes_mask[proto_id].tolist() == 0:
                continue

            words = [itos[int(token)] for token in tokens_list]
            words = [w for w in words if w not in ['<START>', '<END>', '<pad>', '/><br', '<unk>']]
            words = [w if w[:2] != '/>' else w[2:] for w in words]
            words = [w.replace('<br','').replace('\\n', ' ').replace('  ',' ') for w in words]

            indexes_remove = []
            for i, w in enumerate(words):
                if w in words_remove:
                    indexes_remove.append(i)
                else:
                    break
            for i, w in enumerate(reversed(words)):
                if w in words_remove:
                    indexes_remove.append(len(words)-1-i)
                else:
                    break

            words = [w if i in indexes_remove else f'\\textbf{{{w}}}'
                     for i, w in enumerate(words)]
            phrase  = ' '.join(words)

            df_data.append([phrase, data_visualizer.fc_weights[proto_id]])

        df = pd.DataFrame(df_data,columns=['Prototype','Weight']).sort_values('Weight', ascending=False)
        df['Weight'] = df['Weight'].round(3)
        print(df.to_latex(index=False, escape=False))


\begin{tabular}{lr}
\toprule
                                                                                                  Prototype &  Weight \\
\midrule
                                                       for me this is a \textbf{pleasant} \textbf{surprise} &   1.229 \\
 \textbf{danson} \textbf{and} \textbf{whoopie} \textbf{goldberg} \textbf{is} \textbf{an} \textbf{awesomely} &   0.849 \\
                                the \textbf{movie} \textbf{is} \textbf{also} \textbf{very} \textbf{stylish} &   0.845 \\
                                 this \textbf{movie} \textbf{is} \textbf{just} \textbf{a} \textbf{waste} of &  -0.660 \\
             \textbf{children} \textbf{is} \textbf{pathetic} \textbf{and} \textbf{the} \textbf{attempts} to &  -0.702 \\
                           it was \textbf{laughable} \textbf{i} \textbf{could} \textbf{n't} \textbf{resist} &  -0.953 \\
\bottomrule
\end{tabular}



\begin{tabular}{lr}
\toprule
                                                                                                              Prototype &  Weight \\
\midrule
                                   's \textbf{life} \textbf{is} \textbf{a} \textbf{really} \textbf{good} \textbf{piece} &   0.872 \\
                                               this is a \textbf{pretty} \textbf{good} \textbf{ensemble} \textbf{piece} &   0.653 \\
                                                                          it 's a very \textbf{nice} \textbf{movie} and &   0.571 \\
                                        the \textbf{worst} \textbf{horror} \textbf{movie} \textbf{ever} \textbf{put} to &  -0.445 \\
                                                                               \textbf{waste} \textbf{of} \textbf{time} &  -0.460 \\
                             \textbf{script} \textbf{but} \textbf{this} \textbf{was} \textbf{really} \textbf{awful} the &  -0.839 \\
                       \textbf{

\begin{tabular}{lr}
\toprule
                                                                                               Prototype &  Weight \\
\midrule
                     \textbf{also} \textbf{is} \textbf{excellent} \textbf{and} \textbf{innovative.} this &   0.798 \\
 are \textbf{still} \textbf{producing} \textbf{good} \textbf{films} \textbf{highly} \textbf{recommended} &   0.684 \\
          will \textbf{delight} \textbf{the} \textbf{whole} \textbf{family} \textbf{it} \textbf{manages} &   0.589 \\
                  it what \textbf{really} \textbf{makes} \textbf{it} \textbf{all} \textbf{unforgettable} &   0.278 \\
              the \textbf{world} \textbf{it} \textbf{is} \textbf{an} \textbf{inspiring} \textbf{message} &   0.118 \\
                     \textbf{worst} \textbf{film} \textbf{i} \textbf{have} \textbf{ever} \textbf{seen} i &  -0.363 \\
        \textbf{n't} \textbf{waste} \textbf{your} \textbf{time} \textbf{with} \textbf{the} \textbf{plot} &  -0.443 \\
       \textbf{wor

\begin{tabular}{lr}
\toprule
                                                                                                          Prototype &  Weight \\
\midrule
                 its \textbf{narration} \textbf{with} \textbf{memorable} \textbf{soundtrack} \textbf{especialy} the &   0.822 \\
 \textbf{score} \textbf{was} \textbf{especially} \textbf{memorable} \textbf{and} \textbf{emotion} \textbf{stirring} &   0.623 \\
          \textbf{lush} \textbf{and} \textbf{romantic} \textbf{which} \textbf{made} \textbf{it} \textbf{acceptable} &   0.616 \\
                    \textbf{great} \textbf{the} \textbf{story} \textbf{is} \textbf{nice} \textbf{it} \textbf{tells} &   0.440 \\
                                     her \textbf{best} \textbf{to} \textbf{resist} \textbf{his} \textbf{powers} but &   0.080 \\
                                           about it just \textbf{plain} \textbf{crap} \textbf{and} \textbf{whoever} &  -0.278 \\
                         \textbf{ok} \textbf{and} \textbf{b

\begin{tabular}{lr}
\toprule
                                                                                                       Prototype &  Weight \\
\midrule
 \textbf{atmosphere} \textbf{and} \textbf{lend} \textbf{it} \textbf{an} \textbf{almost} \textbf{expressionistic} &   0.867 \\
        \textbf{smooth} \textbf{criminal} \textbf{which} \textbf{is} \textbf{absolutely} \textbf{fantastic} with &   0.700 \\
                             and \textbf{highly} \textbf{recommend} \textbf{it!} \textbf{the} \textbf{directing} &   0.619 \\
                              but \textbf{overall} \textbf{it} \textbf{was} \textbf{very} \textbf{enjoyable} and &   0.524 \\
                                     the \textbf{story} \textbf{has} \textbf{lots} \textbf{of} \textbf{holes} in &   0.144 \\
                          the \textbf{characters} \textbf{are} \textbf{believable} \textbf{the} \textbf{plot} is &  -0.018 \\
                                                           a \textbf{story} \tex

...

\begin{tabular}{lr}
\toprule
                                                                                                Prototype &  Weight \\
\midrule
                         very \textbf{clean} \textbf{with} \textbf{lots} \textbf{of} \textbf{sunbeds} and &   0.721 \\
                  they are \textbf{good} \textbf{at} \textbf{keeping} \textbf{everything} \textbf{nicely} &   0.605 \\
                                            with \textbf{chanel} \textbf{our} \textbf{new} \textbf{puppy} &   0.477 \\
      \textbf{amazing} \textbf{food} \textbf{and} \textbf{sites} \textbf{to} \textbf{visit} \textbf{dear} &   0.438 \\
      \textbf{able} \textbf{to} \textbf{retrieve} \textbf{my} \textbf{beach} \textbf{chairs} \textbf{etc} &   0.362 \\
                            will were \textbf{clean} \textbf{and} \textbf{very} \textbf{comfortable} .... &   0.243 \\
                      they 're \textbf{doggie} \textbf{friendly} \textbf{which} \textbf{is} \textbf{cute} &   0.194 \\
          

\begin{tabular}{lr}
\toprule
                                                                                                      Prototype &  Weight \\
\midrule
                               and they had \textbf{homemade} \textbf{chocolate} \textbf{chip} \textbf{cookies} &   1.042 \\
                   and \textbf{quiet} \textbf{the} \textbf{bed} \textbf{was} \textbf{very} \textbf{comfortable} &   0.827 \\
                                                          a \textbf{beautiful} \textbf{historic} \textbf{hotel} &   0.634 \\
                                                                  very \textbf{pleasant} \textbf{stay} here the &   0.453 \\
         \textbf{lit} \textbf{candle} \textbf{the} \textbf{mariachi} \textbf{guys} \textbf{sang} \textbf{happy} &   0.404 \\
        \textbf{currency} \textbf{first} \textbf{and} \textbf{this} \textbf{created} \textbf{a} \textbf{slight} &   0.296 \\
                       some \textbf{upgrading} \textbf{to} \textbf{our} \textbf{busines

\begin{tabular}{lr}
\toprule
                                                                                                   Prototype &  Weight \\
\midrule
                                                                            \textbf{great} \textbf{hotel} in &   0.807 \\
      \textbf{area} \textbf{the} \textbf{showers} \textbf{and} \textbf{little} \textbf{lounge} \textbf{area} &   0.738 \\
                                                                              \textbf{great} \textbf{stay} i &   0.647 \\
                         have \textbf{always} \textbf{enjoyed} \textbf{staying} \textbf{here} \textbf{clean} &   0.563 \\
                                                     had a \textbf{wonderful} \textbf{time} \textbf{staying} &   0.282 \\
                                               all a \textbf{great} \textbf{place} \textbf{to} \textbf{stay} &   0.150 \\
                                a few \textbf{days} \textbf{in} \textbf{napa} \textbf{valley} \textbf{found}

\begin{tabular}{lr}
\toprule
                                                                                                           Prototype &  Weight \\
\midrule
 \textbf{welcoming} \textbf{you} \textbf{back} \textbf{soon} \textbf{sincerely} \textbf{alex} \textbf{blancogeneral} &   0.762 \\
       \textbf{many} \textbf{family} \textbf{members} \textbf{attending} \textbf{a} \textbf{family} \textbf{reunion} &   0.696 \\
                   \textbf{future} \textbf{until} \textbf{then} \textbf{happy} \textbf{travels!sarah} \textbf{junge} &   0.581 \\
                                                                                           this was a \textbf{great} &   0.381 \\
                                                \textbf{next} \textbf{2} \textbf{weeks} \textbf{spoke} with ... more &   0.031 \\
                                                     the \textbf{room} \textbf{was} \textbf{very} \textbf{clean} but &   0.016 \\
                                    in my \textbf{co

\begin{tabular}{lr}
\toprule
                                                                                                        Prototype &  Weight \\
\midrule
                   very \textbf{convienent} \textbf{to} \textbf{the} \textbf{turner} \textbf{falls} \textbf{park} &   0.940 \\
                          the very \textbf{warm} \textbf{welcome} \textbf{and} \textbf{service} \textbf{provided} &   0.912 \\
                     and \textbf{everyone} \textbf{was} \textbf{very} \textbf{nice} \textbf{and} \textbf{helpful} &   0.735 \\
 \textbf{immediately} \textbf{the} \textbf{concierge} \textbf{lounge} \textbf{was} \textbf{well} \textbf{staffed} &   0.619 \\
                           your \textbf{toiletries} \textbf{on} \textbf{the} \textbf{bathroom} \textbf{space} was &   0.193 \\
    \textbf{attending} \textbf{the} \textbf{international} \textbf{rotary} \textbf{convention} \textbf{sorry} for &   0.185 \\
                                         in for the \textbf{trip} \textbf